In [2]:
import os

from dotenv import load_dotenv
from langchain_openai import OpenAI, AzureOpenAI, AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import AgentType, initialize_agent, load_tools

### Build a LLM application with LangChain Framework

#### 1. Set the environment variables.

In [3]:
load_dotenv()

os.environ["GOOGLE_CSE_ID"] = os.getenv("GOOGLE_SEARCH_ENGINE_ID")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_SEARCH_API_KEY")

#### 2. Use the model directly.
Reference: https://python.langchain.com/v0.2/docs/tutorials/llm_chain/

In [4]:
# Instantiate a chat model and invoke it with the messages.
# llm = AzureOpenAI(deployment_name="gpt-35-turbo")
# llm.invoke("Tell me a joke")

In [5]:
user_prompt = "回答時，使用繁體中文和台灣詞語解釋, 問題: NBA 的英文全名是？"

model = AzureChatOpenAI(deployment_name="gpt-35-turbo-120")
messages = [HumanMessage(content=user_prompt)]
completion = model.invoke(messages)
completion.content

'NBA 的英文全名是 National Basketball Association，中文為「美國職業籃球聯盟」。它是美國最高級別的職業籃球聯賽，也是全球最具影響力和知名度的籃球聯賽之一。'

In [7]:
model.predict(user_prompt)

'NBA 的英文全名是 National Basketball Association，翻譯成繁體中文是「美國職業籃球聯賽」，是美國最高級別的職業籃球聯賽。'

#### 3. Add prompt templates.
Use to create flexible templated prompts for chat models.

In [8]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Translate the following into {language}:"),
    ("user", "{text}"),
])
messages = prompt_template.invoke({
    "language": "英文",
    "text": "你吃飽了嗎？"
})
completion = model.invoke(messages)
completion.content

'Have you eaten yet?'

#### 4. Chain together components with LCEL.

In [9]:
chain = prompt_template | model | StrOutputParser()
chain.invoke({
    "language": "英文",
    "text": "你吃飽了嗎？"
})

'Have you eaten yet?'

#### 5.  Incorporate a memory of previous messages to sustain a stateful conversation.
Reference: https://python.langchain.com/v0.2/docs/tutorials/chatbot/#message-history

In [10]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


prompt_template = ChatPromptTemplate.from_messages([
    ("system", "以下是使用者與 AI 之間友好的對話， AI 很健談且能根據上下文提供具體細節，如果 AI 不知道問題的答案會如實說不知道"),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
])

chain = prompt_template | model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_session_history,
    history_messages_key="chat_history",
)


session_id = "my_session_id"

while True:
    user_prompt = input("User:")
    if not user_prompt.strip():
        break
    print(f'User: {user_prompt}')

    assistant_prompt = wrapped_chain.invoke(
        {"input": user_prompt},
        config={"configurable": {"session_id": session_id}},
    )
    print(f'AI: {assistant_prompt}')

User: 你好，我叫寶寶，今年三歲，最喜歡吃蘋果。
AI: 你好，寶寶！很高興認識你。三歲是個很可愛的年齡呢！喜歡吃蘋果是一件很健康的事情，蘋果含有豐富的維生素和纖維，對身體很有益處呢。你喜歡吃紅蘋果還是綠蘋果呢？
User: 我喜歡紅蘋果。我現在想吃蛋糕，你有推薦的口味嗎？
AI: 喜歡紅蘋果的你真是個聰明的寶寶呢！紅蘋果的口感和甜度都很適合小朋友喜歡呢。

關於蛋糕的推薦，有很多口味可以選擇哦！你喜歡巧克力、草莓、香蕉還是檸檬味的呢？每一種口味都有自己獨特的風味，我可以為你介紹更多細節，幫助你做出選擇。
User: 那我要插上幾歲的蠟燭呢？
AI: 嗯，通常在生日蛋糕上插上的蠟燭數字代表著你的年齡。因為你告訴我你今年三歲，所以我們可以在蛋糕上插上數字3的蠟燭，這樣就可以慶祝你的三歲生日囉！你有想要特別的蠟燭顏色或形狀嗎？


In [11]:
chat_history = get_session_history(session_id)
chat_history.messages

[HumanMessage(content='你好，我叫寶寶，今年三歲，最喜歡吃蘋果。'),
 AIMessage(content='你好，寶寶！很高興認識你。三歲是個很可愛的年齡呢！喜歡吃蘋果是一件很健康的事情，蘋果含有豐富的維生素和纖維，對身體很有益處呢。你喜歡吃紅蘋果還是綠蘋果呢？'),
 HumanMessage(content='我喜歡紅蘋果。我現在想吃蛋糕，你有推薦的口味嗎？'),
 AIMessage(content='喜歡紅蘋果的你真是個聰明的寶寶呢！紅蘋果的口感和甜度都很適合小朋友喜歡呢。\n\n關於蛋糕的推薦，有很多口味可以選擇哦！你喜歡巧克力、草莓、香蕉還是檸檬味的呢？每一種口味都有自己獨特的風味，我可以為你介紹更多細節，幫助你做出選擇。'),
 HumanMessage(content='那我要插上幾歲的蠟燭呢？'),
 AIMessage(content='嗯，通常在生日蛋糕上插上的蠟燭數字代表著你的年齡。因為你告訴我你今年三歲，所以我們可以在蛋糕上插上數字3的蠟燭，這樣就可以慶祝你的三歲生日囉！你有想要特別的蠟燭顏色或形狀嗎？')]

#### 6. Use tools or components.

Useful for when you need to ...
- search the web, use the "[google-search](https://python.langchain.com/v0.2/docs/integrations/tools/google_search/)" tool
- solve math problems, use the "llm-math" tool

Reference: https://python.langchain.com/v0.2/docs/integrations/tools/search_tools/#googlesearchapiwrapper

In [14]:
user_prompt = "請問 2023 金馬獎影帝是誰？"

agent = initialize_agent(
    tools=load_tools(tool_names=["google-search"], llm=model),
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
assistant_prompt = agent.run(user_prompt)

print(f'Prompt: {user_prompt}')
print(f'Completion: {assistant_prompt}')



> Entering new AgentExecutor chain...
I don't have the answer to this question in my existing knowledge. I should use the google_search tool to find the information.
Action: google_search
Action Input: "2023 金馬獎影帝 是誰"
Observation: Nov 25, 2023 ... 金馬獎女主角林品彤！金馬影帝吳慷仁擊敗阮經天、許光漢和林柏宏。金馬60林品彤成為最年輕金馬影后打敗陸小芬！金馬60得獎 ... 是電影《老狐狸》，抱回金馬 ... Nov 25, 2023 ... 【2023 金馬獎】金馬60 最新得獎名單一次看：吳慷仁首次入圍就奪影帝、12 歲林品彤成最年輕影后！ · 洪昰顥《周處除三害》（得獎） · 邱立偉、王俊雄《八戒》 ... Mar 1, 2024 ... 第60屆2023金馬獎頒獎典禮將於11月25日（六）於台北國父紀念館舉行，而2023金馬獎入圍名單也將熱騰騰曝光，其中備受矚目的，則是許光漢與林柏宏都以《關於 ... 金馬獎2023》金馬60得獎名單：影帝吳慷仁、12歲影后林品彤 · 入圍暨得獎名單 · 典禮時間地點 · 直播／轉播平台 · 金馬獎主持人 · 表演嘉賓 · 頒獎嘉賓. Nov 26, 2023 ... 2023金馬獎頒獎典禮11 月25 日在國父紀念館盛大舉行，60 屆金馬獎 ... 是兩大男主角許光漢、林柏宏雙雙提名金馬影帝，競爭激烈，究竟獎落誰家 獲獎者有「金馬影帝」之讚譽。獎項於1962年第1屆金馬獎首度頒發，獲獎者是主演《手槍》的王引。目前，獎項提名人選由評審委員透過公開討論、不記名投票方式產生，獲獎 ... Nov 26, 2023 ... 2023金馬獎（金馬60）順利落幕，金馬影帝吳慷仁、金馬影后林品彤、女配角方志友等都是首次入圍就拿獎。典禮後，本屆評審團主席李安導演、金馬執委會 ... Nov 25, 2023 ... 恭喜吳慷仁獲得2023年金馬影帝！今年8月吳慷仁以《富都青年》奪得第17屆西寧FIRST青年電影展「主競賽單元最佳演員獎」， ... Nov 26, 2023 ... 最佳男配角為

In [16]:
user_prompt = "請問  1 + 4 = ?"

agent = initialize_agent(
    tools=load_tools(tool_names=['llm-math'], llm=model),
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
assistant_prompt = agent.run(user_prompt)

print(f'Prompt: {user_prompt}')
print(f'Completion: {assistant_prompt}')



> Entering new AgentExecutor chain...
I should use the Calculator tool to add 1 and 4.
Action: Calculator
Action Input: 1 + 4
Observation: Answer: 5
Thought:The sum of 1 and 4 is 5.
Final Answer: 5

> Finished chain.
Prompt: 請問  1 + 4 = ?
Completion: 5
